# K-Means Anchors Ratios Tutorial

In [1]:
import json
import numpy as np
import os

%load_ext autoreload
%autoreload 2

## Get dataset instances

In [2]:
INSTANCES_PATH = "./annotations/instances_train.json"
with open(INSTANCES_PATH) as f:
    instances = json.load(f)

## Get optimal anchors ratios

In [3]:
## change the following parameters according to your model:

# EfficientDetD{PHI}
PHI = 2  # for another efficientdet change only this, e.g. PHI = 3 for D3

input_sizes = [512, 640, 768, 896, 1024, 1280, 1280, 1536, 1536]
pyramid_levels = [5, 5, 5, 5, 5, 5, 5, 5, 6]
anchor_scale = [4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 5.0, 4.0]

scale = anchor_scale[PHI]
strides = 2 ** np.arange(3, pyramid_levels[PHI] + 3)
scales = np.array([2 ** 0, 2 ** (1.0 / 3.0), 2 ** (2.0 / 3.0)])

INPUT_SIZE = input_sizes[PHI]
ANCHORS_SIZES = (scale * scales * strides[:, np.newaxis]).flatten().tolist()
ANCHORS_SIZES

[32.0,
 40.31747359663594,
 50.79683366298238,
 64.0,
 80.63494719327188,
 101.59366732596476,
 128.0,
 161.26989438654377,
 203.18733465192952,
 256.0,
 322.53978877308754,
 406.37466930385904,
 512.0,
 645.0795775461751,
 812.7493386077181]

In [4]:
from kmeans_anchors_ratios import get_optimal_anchors_ratios

anchors_ratios = get_optimal_anchors_ratios(
    instances,
    anchors_sizes=ANCHORS_SIZES,
    input_size=INPUT_SIZE,
    normalizes_bboxes=True,
    num_runs=3,
    num_anchors_ratios=3,
    max_iter=300,
    iou_threshold=0.5,
    min_size=0,
    decimals=1,
    default_anchors_ratios=[(0.7, 1.4), (1.0, 1.0), (1.4, 0.7)]
)

[03/24 20:50:39] Starting the calculation of the optimal anchors ratios
[03/24 20:50:39] Extracting and preprocessing bounding boxes
[03/24 20:50:39] Discarding 0 bounding boxes with size lower or equal to 0


[03/24 20:50:39] K-Means (3 runs): 100%|██████████████████| 3/3 [00:00<00:00,  3.96it/s]

	Runs avg. IoU: 95.09% ± 0.00% (mean ± std. dev. of 3 runs, 0 skipped)
	Avg. IoU between bboxes and their most similar anchors after norm. them to make their area equal (only ratios matter): 95.09%


[03/24 20:50:41] Default anchors ratios: [(0.7, 1.4), (1.0, 1.0), (1.4, 0.7)]
	Avg. IoU between bboxes and their most similar default anchors, no norm. (both ratios and sizes matter): 86.22%
	Num. bboxes without similar default anchors (IoU < 0.5):  0/80113 (0.00%)
[03/24 20:50:41] K-Means anchors ratios: [(0.9, 1.1), (1.0, 1.0), (1.1, 0.9)]
	Avg. IoU between bboxes and their most similar K-Means anchors, no norm. (both ratios and sizes matter): 88.07%
	Num. bboxes without similar K-Means anchors (IoU < 0.5):  10/80113 (0.01%)
[03/24 20:50:41] Default anchors have an IoU < 50% with bboxes in 0.01% less cases than the K-Means anchors, you should consider stick with them


In [5]:
anchors_ratios

[(0.9, 1.1), (1.0, 1.0), (1.1, 0.9)]

## Generate anchors given ratios and sizes

In [6]:
from kmeans_anchors_ratios import generate_anchors_given_ratios_and_sizes


anchors = generate_anchors_given_ratios_and_sizes(anchors_ratios, ANCHORS_SIZES)
print("Anchors:")
print(anchors)

Anchors:
[[ 28.8         35.2       ]
 [ 32.          32.        ]
 [ 35.2         28.8       ]
 [ 36.28572624  44.34922096]
 [ 40.3174736   40.3174736 ]
 [ 44.34922096  36.28572624]
 [ 45.7171503   55.87651703]
 [ 50.79683366  50.79683366]
 [ 55.87651703  45.7171503 ]
 [ 57.6         70.4       ]
 [ 64.          64.        ]
 [ 70.4         57.6       ]
 [ 72.57145247  88.69844191]
 [ 80.63494719  80.63494719]
 [ 88.69844191  72.57145247]
 [ 91.43430059 111.75303406]
 [101.59366733 101.59366733]
 [111.75303406  91.43430059]
 [115.2        140.8       ]
 [128.         128.        ]
 [140.8        115.2       ]
 [145.14290495 177.39688383]
 [161.26989439 161.26989439]
 [177.39688383 145.14290495]
 [182.86860119 223.50606812]
 [203.18733465 203.18733465]
 [223.50606812 182.86860119]
 [230.4        281.6       ]
 [256.         256.        ]
 [281.6        230.4       ]
 [290.2858099  354.79376765]
 [322.53978877 322.53978877]
 [354.79376765 290.2858099 ]
 [365.73720237 447.01213623]
 [406

## Get bounding boxes adapted to the input size

In [7]:
from kmeans_anchors_ratios import get_bboxes_adapted_to_input_size


resized_bboxes = get_bboxes_adapted_to_input_size(instances, input_size=INPUT_SIZE)
resized_bboxes = resized_bboxes[resized_bboxes.prod(axis=1) > 0]  # remove 0 size
print("Bounding boxes adapted to the input size (first 5):")
print(resized_bboxes[:5])

Bounding boxes adapted to the input size (first 5):
[[48.   48.48]
 [48.   49.44]
 [51.84 53.28]
 [48.96 49.92]
 [58.56 54.72]]


## Get the avg. IoU between the bounding boxes and their closest anchors

In [8]:
from kmeans_anchors_ratios import average_iou


avg_iou = average_iou(resized_bboxes, anchors)
print(f"Avg. IoU: {100 * avg_iou:.2f}%")

Avg. IoU: 88.07%


## Get annotations whose bounding boxes don't have similar anchors

In [9]:
from kmeans_anchors_ratios import get_annotations_without_similar_anchors


annotations = get_annotations_without_similar_anchors(
    instances,
    anchors_ratios,
    anchors_sizes=ANCHORS_SIZES,
    input_size=INPUT_SIZE,
    iou_threshold=0.5,
    min_size=0,
)

bboxes = [ann["bbox"][-2:] for ann in annotations]  # widths and heights
print("Bounding boxes without similar anchors (first 5):")
print(bboxes[:5])

instances_without_similar_anchors = instances.copy()
instances_without_similar_anchors["annotations"] = annotations
resized_bboxes = get_bboxes_adapted_to_input_size(instances_without_similar_anchors, input_size=INPUT_SIZE)
print("Bounding boxes without similar anchors adapted to the input size (first 5):")
print(resized_bboxes[:5])

Bounding boxes without similar anchors (first 5):
[[249, 88], [191, 67], [207, 69], [62, 175], [224, 75]]
Bounding boxes without similar anchors adapted to the input size (first 5):
[[119.52  42.24]
 [ 91.68  32.16]
 [ 99.36  33.12]
 [ 29.76  84.  ]
 [107.52  36.  ]]
